# Solar and Wind Curtailment

Renewables like wind and solar regularly produce energy in excess of demand. In order to keep supply and demand balanced on the grid, the result is "curtailment", or purposefully reducing output.

In this notebook, we'll walk through accessing the curtailment data for CAISO

In [ ]:
import gridstatus
import pandas as pd
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
iso = gridstatus.CAISO()

## Get Curtailment Data

First, we will query for curtailment data. CAISO publishes curtailment data starting on June 30, 2016. We use the `save_to` parameter to save the data locally, so it is easier to reload later.

In [ ]:
df = iso.get_curtailment(
    start="Jan 1, 2020", end="Nov 30, 2022", save_to="curtailment/"
)

we can easily reload the data in the curtailment folder like this. By default it loads with UTC timezone unless we specify otherwise.

In [ ]:
df = gridstatus.load_folder("curtailment", time_zone=gridstatus.CAISO.default_timezone)
df

next, let's reformat the data to make it easier to work with


In [ ]:
df["Type"] = (
    df["Curtailment Reason"].str.lower().str.capitalize()
    + " "
    + df["Fuel Type"]
    + " Curtailment (MWh)"
)
curtailment = df.pivot_table(
    values="Curtailment (MWh)", index="Time", columns="Type"
).fillna(0)

curtailment["Total Solar Curtailment (MWh)"] = (
    curtailment["Local Solar Curtailment (MWh)"]
    + curtailment["System Solar Curtailment (MWh)"]
)
curtailment["Total Wind Curtailment (MWh)"] = (
    curtailment["Local Wind Curtailment (MWh)"]
    + curtailment["System Wind Curtailment (MWh)"]
)
curtailment["Total Curtailment (MWh)"] = (
    curtailment["Total Solar Curtailment (MWh)"]
    + curtailment["Total Wind Curtailment (MWh)"]
)
curtailment.columns.name = None
curtailment = curtailment.resample("1H").sum()
curtailment

## Visualizing Curtailment

### Monthly Curtailment

In [ ]:
monthly = curtailment.resample("M").sum()
monthly["Month"] = monthly.index.month
monthly["Year"] = monthly.index.year

fig = px.bar(
    monthly,
    x=monthly.index,
    y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Monthly Solar and Wind Curtailment in CAISO (MWh)",
)

# legend upper left corner
fig.update_layout(
    legend=dict(
        orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0, title_text=None
    )
)
fig.update_yaxes(title_text="Curtailment (MWh)")
fig.show("svg", width=1200, height=600)

### Average Hourly Curtailment

In [ ]:
avg_hourly = curtailment.groupby(curtailment.index.hour).mean()
fig = px.line(
    avg_hourly,
    x=avg_hourly.index,
    y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Average Hourly Solar and Wind Curtailment in CAISO (MWh)",
)
fig.update_yaxes(title_text="Curtailment (MWh)")
fig.show("svg", width=1200, height=600)

In [ ]:
curtailment["Year"] = curtailment.index.year
yearly_sum = curtailment.groupby("Year").sum()
index = yearly_sum.index.astype(str).tolist()
index[-1] = "2022 YTD"
yearly_sum.index = index

fig = px.bar(
    yearly_sum,
    x=yearly_sum.index,
    y=["Total Solar Curtailment (MWh)", "Total Wind Curtailment (MWh)"],
    title="Total Solar and Wind Curtailment in CAISO (MWh)",
)
fig.update_layout(
    legend=dict(yanchor="bottom", y=0.9, xanchor="left", x=0, title_text=None)
)
fig.update_yaxes(title_text="Curtailment (MWh)")
fig.update_xaxes(title_text="Year")
fig.show("svg", width=1200, height=600)